In [1]:
#Importing the libraries
import requests
import base64
import io
from PIL import Image
import os
import glob
import numpy as np
import string
from pdf2image import convert_from_path
from pathlib import Path
import pytesseract
from datetime import date
import pandas as pd
import cv2
import matplotlib.pyplot as plt 
today = date.today()
#erase the data and remove the dir
import shutil



#Function for input pdf file

def NEW_rfp_image_segmentation(pdf_file):
    
    lab_dict = {0:'Footer', 1:'H1', 2:'H2', 3:'H3', 4:'H4', 5:'H5', 6:'Header', 7:'Image', 8:'P1', 9:'Table', 10:'table'}
    #lab_dict = {0:'Footer', 1:'H1', 2:'H2'}
    today = date.today()
    def Sort(sub_li):
        sub_li.sort(key = lambda x: x[2])
        return sub_li

    # getting the name of the file that came through API
    rfxpdf = pdf_file
    filename = os.path.basename(str(rfxpdf))
    print(filename)
    
    #Creating folder, if not exists!
    folder_name = 'segmentation_run_data/rfx_pdf_folder/'+str(today)+ '/'+filename[:-4] + '/'
    predict_folder = './segmentation_run_data/runs/detect/'+str(today)+ '/'+filename[:-4] + '/'
    
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    else :
        shutil.rmtree(folder_name)
        os.makedirs(folder_name)
        
        
    if not os.path.exists(predict_folder):
        os.makedirs(predict_folder)
    else :
        shutil.rmtree(predict_folder)
        os.makedirs(predict_folder)   
    

# converting the pages of the pdf to images and saving in the directory created above
    
    images = convert_from_path(rfxpdf, 500)
    for i, image in enumerate(images):
        fname = folder_name+str(i)+'.jpg'
        #print(fname)
        image.save(fname, "JPEG")
 
 #Reading images from glob (global path)
    from glob import glob
    tp=glob(r'segmentation_run_data/rfx_pdf_folder/'+str(today)+ '/'+filename[:-4] + '/*.jpg')
    
    
#Importing ROBOFLOW API
    from roboflow import Roboflow
#     # obtaining your API key: https://docs.roboflow.com/rest-api#obtaining-your-api-key
    rf = Roboflow(api_key="GxPz62ep4pVFabB6fUGl")

    workspace = rf.workspace('paulsolomon')
    project = workspace.project('sevensets')
    version = project.version(1)
    model = version.model
    
#Processing on image and storing it in output_path: folder assign
    #label_count=0
    #image_count=0
    name=''
    for image in tp:
        print(image)
        
        name=image.split('/')[-1]                         
        #print(name)
        without_ext_img=name.rsplit('.',maxsplit=1)[0]
        #print(without_ext_img)
        #try:
        prediction = model.predict(image)           
        #except:
        #    print(image,"error")
        #    pass
#         #prediction.plot()
#         #prediction.json()
        
        #try:
        prediction.save(output_path=predict_folder+"predictions{}.jpg".format(without_ext_img))
        #except:
        #    print("error")
        #    pass
#         #prediction.save(output_path="./segmentation_run_data/runs/detect/predictions{}.jpg".format(image_count))
#         #output_path="abc/" + "predictions{}.jpg".format(image_count)
#         #prediction.save(output_path=output_path)
#         #image_count+=1
        
    
#Retreiving the cordinates from jsons response of API
        count=0
        b=""
        a=prediction.json()
        size=(len(a['predictions']))
        for i in range(0,size):
            if a['predictions'][i]['class']:
                label=a['predictions'][i]['class']
                #print(label)
                for k,v in lab_dict.items():
                    if v==label:
                        a['predictions'][i]['class']=k


        while count < len (a['predictions']):
          #print(a['predictions'][count]['x'])
          b+=str(a['predictions'][count]['x'])+" "+str(a['predictions'][count]['y'])+" "+str(a['predictions'][count]['width'])+" "+str(a['predictions'][count]['height'])+" "+str(a['predictions'][count]['class'])+"\n"
          #print(count)
          count+=1
        
#Creating folder to store the labels
        if not os.path.exists(predict_folder+"/labels"):
            os.makedirs(predict_folder+"/labels")

        with open(predict_folder+"./labels/predictions{}.txt".format(without_ext_img),"w") as d:
               d.write(b) 
        #label_count+=1
        
        
#Extracting the text from images
    for image in tp:
        pages = {}
        for predicted_image in os.listdir(folder_name):
            print('Tesseract job started')
            count = 1
            if predicted_image.endswith('.jpg'):
                stemp = Path(predicted_image).stem
                #print(stemp)
                #print only name of file without extensions
                img = cv2.imread(folder_name+predicted_image)     #read the image in np.array form
                dh, dw, _ = img.shape                                   
                label_path = predict_folder +'/labels/predictions'+str(stemp)+'.txt'
                #print(os.path.exists(label_path),label_path)
                
                if os.path.exists(label_path):
                    fl = open(label_path, 'r')
                    data = fl.readlines()
                    fl.close() 

# Getting the yolo coordinates to bounding box coordinates.
                    boxes = []
                    for dt in data:
                        #lab, x, y, w, h = map(float, dt.split(' '))
                        x, y, w, h, lab = map(float, dt.split(' '))

                        l = int(x - w // 2)
                        r = int(x + w // 2)
                        t = int(y - h // 2)
                        b = int(y + h // 2)
                        if l < 0:
                            l = 0
                        if r > dw - 1:
                            r = dw - 1
                        if t < 0:
                            t = 0
                        if b > dh - 1:
                            b = dh - 1
                        cv2.rectangle(img, (l, t), (r, b), (255, 0, 0), 1)
                        boxes.append([lab,l, t, r, b])
                    boxes = Sort(boxes)
                    #print(boxes)

# for dims,img,item in bounding_boxes:
                    abc = string.ascii_lowercase + string.ascii_uppercase
                    crops = {}
                    i = 0
                    for box in boxes:
                        #print(box)
                        lab,l,t,r,b = box
                        crops[str(int(lab))+abc[i]] = img[t-5:b+5,]
                        #plt.imshow(img[t-5:b+5,])
                        #print(crops)
                        #print(lab,l,t,r,b)
                        #print(img[t:b,].shape)
                        cv2.rectangle(img, (l, t), (r, b), (0, 0, 255), 1)
                        i+=1

                    #plt.imshow(img)
                    #plt.show()
# Extracing the text using pytesseract and placing the text in order for forming json structure.
                    jsons = []
                    count_i = 1
                    tag = []
                    temp_dx=[]
                    df = pd.DataFrame(columns=['A','B'])
                    for k,v in crops.items():

                        extracted = pytesseract.image_to_string(crops[k])
                        temp_dx.append(extracted)

                        name = lab_dict[int(k[0])]
                        #print(name)
                        #print(type(name))
                        sortorder = count_i
                        dct = {"name": name,
                                "extracted": extracted,
                                "sortOrder": count_i}
                        count_i+=1
                        jsons.append(dct)
                        tag.append([name,extracted])
                        d1 =pd.DataFrame({'A':name,'B':extracted},index=[0])
                        #d2 = {'B': str(extracted)}
                        pd.concat([df,d1], ignore_index = True)
                        #df.append(d2, ignore_index = True)
                    x =pd.DataFrame(tag,columns=["name","extracted"])  
                    x.to_csv("extracted.csv")
                    df.to_csv("ex2.csv")


                    pages['Page: '+str(stemp)] = jsons

                else:
                    pages['Page: '+str(stemp)] = 'NA'
                print('page done' + str(stemp))
                count+=1

        return pages 



NEW_rfp_image_segmentation("IT_Consulting_Services_Page_01-04-withTable.pdf") 


IT_Consulting_Services_Page_01-04-withTable.pdf
loading Roboflow workspace...
loading Roboflow project...
segmentation_run_data/rfx_pdf_folder/2022-09-23/IT_Consulting_Services_Page_01-04-withTable\0.jpg
segmentation_run_data/rfx_pdf_folder/2022-09-23/IT_Consulting_Services_Page_01-04-withTable\1.jpg
segmentation_run_data/rfx_pdf_folder/2022-09-23/IT_Consulting_Services_Page_01-04-withTable\2.jpg
segmentation_run_data/rfx_pdf_folder/2022-09-23/IT_Consulting_Services_Page_01-04-withTable\3.jpg
Tesseract job started
page done0
Tesseract job started
page done1
Tesseract job started
page done2
Tesseract job started
page done3


{'Page: 0': [{'name': 'H1', 'extracted': 'INTRODUCTION:\n\n', 'sortOrder': 1},
  {'name': 'P1',
   'extracted': 'Four (4) copies of your proposal are required. In addition, please provide one (1) CD that contains an electronic copy of\nyour proposal in MS WORD 6.0 or PDF FORMAT. If brochures or other supportive documents are requested, then it is\nrequired that 4 sets be submitted with your proposal.\n\nPayment for work performed or goods sold to Larimer County can be expected within 30 days after receipt of the invoice\nand satisfactory acceptance from the department receiving the service or goods.\n\n',
   'sortOrder': 2},
  {'name': 'P1',
   'extracted': 'The Board of Larimer County Commissioners will be receiving sealed proposals at the office of the Purchasing Director,\n2555 Midpoint Drive, Suite B, Fort Collins, Colorado, up to 2PM (our clock), Wednesday May 7, 2008, at which time\nproposals will be recorded, but not publicly opened, to consider contracting for information techn

In [ ]:
nnnnnnnnnnnnnn dhrtgjhtetyj

In [15]:
#Importing ROBOFLOW API
from roboflow import Roboflow
#     # obtaining your API key: https://docs.roboflow.com/rest-api#obtaining-your-api-key
rf = Roboflow(api_key="GxPz62ep4pVFabB6fUGl")

workspace = rf.workspace('paulsolomon')
project = workspace.project('sevensets')
version = project.version(1)
model = version.model
model.predict(r"C:\Users\prash\Desktop\Paul Soloman\segmentation_run_data\rfx_pdf_folder\2022-09-13\5 App Developer Resource RFP\5.jpg")


loading Roboflow workspace...
loading Roboflow project...


Exception: {"message":"Request Entity Too Large"}